In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import xgboost as xgb

In [2]:
from sklearn.datasets import load_digits

In [3]:
from sklearn.cross_validation import train_test_split

In [5]:
digits = load_digits(2)

In [6]:
X = digits['data']
y = digits['target']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [8]:
clf1 = xgb.XGBClassifier()

In [10]:
clf1.fit(X_train, y_train, early_stopping_rounds=5, eval_metric='auc',
         eval_set=[(X_test, y_test)])

[0]	validation_0-auc:0.999497
Will train until validation_0-auc hasn't improved in 5 rounds.
[1]	validation_0-auc:0.999497
[2]	validation_0-auc:0.999497
[3]	validation_0-auc:0.999749
[4]	validation_0-auc:0.999749
[5]	validation_0-auc:0.999749
[6]	validation_0-auc:0.999749
[7]	validation_0-auc:0.999749
[8]	validation_0-auc:0.999749
Stopping. Best iteration:
[3]	validation_0-auc:0.999749



XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [11]:
clf1.best_score

0.999749

In [12]:
clf2 = xgb.XGBClassifier()
clf2.fit(X_train, y_train, early_stopping_rounds=4, eval_metric="auc",
         eval_set=[(X_test, y_test)])

[0]	validation_0-auc:0.999497
Will train until validation_0-auc hasn't improved in 4 rounds.
[1]	validation_0-auc:0.999497
[2]	validation_0-auc:0.999497
[3]	validation_0-auc:0.999749
[4]	validation_0-auc:0.999749
[5]	validation_0-auc:0.999749
[6]	validation_0-auc:0.999749
[7]	validation_0-auc:0.999749
Stopping. Best iteration:
[3]	validation_0-auc:0.999749



XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [13]:
clf2.best_score

0.999749

In [18]:
# 是否过拟合
clf3 = xgb.XGBClassifier()
clf3.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="auc",
         eval_set=[(X_test, y_test)])

[0]	validation_0-auc:0.999497
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.999497
[2]	validation_0-auc:0.999497
[3]	validation_0-auc:0.999749
[4]	validation_0-auc:0.999749
[5]	validation_0-auc:0.999749
[6]	validation_0-auc:0.999749
[7]	validation_0-auc:0.999749
[8]	validation_0-auc:0.999749
[9]	validation_0-auc:0.999749
[10]	validation_0-auc:1
[11]	validation_0-auc:1
[12]	validation_0-auc:1
[13]	validation_0-auc:1
[14]	validation_0-auc:1
[15]	validation_0-auc:1
[16]	validation_0-auc:1
[17]	validation_0-auc:1
[18]	validation_0-auc:1
[19]	validation_0-auc:1
[20]	validation_0-auc:1
Stopping. Best iteration:
[10]	validation_0-auc:1



XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [19]:
clf3.best_score

1.0

## 交叉验证和early stopping结合起来

### 使用xgboost提供的cv功能，而不是sklearn

In [20]:
dm = xgb.DMatrix(X, label=y)

In [21]:
params = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}

In [23]:
cv = xgb.cv(params, dm, num_boost_round=10, nfold=10, early_stopping_rounds=10)

In [24]:
cv.shape[0]

10

In [25]:
cv = xgb.cv(params, dm, num_boost_round=10, nfold=10, early_stopping_rounds=5)

In [26]:
def evalerror(preds, dtrain):
    from sklearn.metrics import mean_squared_error
    labels = dtrain.get_label()
    return 'rmse', mean_squared_error(labels, preds)

In [29]:
# 使用自定义的评价指标
cv = xgb.cv(params, dm, num_boost_round=10, nfold=10, feval=evalerror, maximize=True, early_stopping_rounds=3)

In [30]:
cv.shape[0]

1